# SimpleITK from R

The R notebooks follow the general structure of the python equivalents.

A few comments to get started:
* R will display a variable when you type the name. If the variable is a SimpleITK image then, by default, R/SimpleITK will attempt to display it with ImageJ. This isn't much good for notebooks or knitted documents, so the "show" method has been replaced by a function using internal R graphics. This isn't necessary for interactive sessions.
* The documentation doesn't exist in the R package yet - be prepared to do some detective work.
* C++ enumerated types are wrapped using strings, rather than integers.
* Error messages are can be very cryptic - it helps to get familiar with the R way of querying methods and objects.
* The object oriented style used by swig is one of the less common ones used in R packages.
* C++ vectors are automatically converted to R vectors.

Let's start by creating an image and accessing information about it.

In [9]:
# Load the SimpleITK library
library(SimpleITK)

imA = Image(256, 128, 64, "sitkInt16")
imB = SimpleITK::Image(64, 64, "sitkFloat32")
imC = Image(c(32, 32), "sitkUInt32")
imRGB = Image(c(128,128), "sitkVectorUInt8", 3)

We've created 3 different (empty) images using slightly differeing syntax to specify the dimensions. We've also specified the pixel type. The range of available pixel types is:

In [5]:
get(".__E___itk__simple__PixelIDValueEnum", pos="package:SimpleITK")

sitkUnknown          sitkUInt8           sitkInt8         sitkUInt16 
                -1                  1                  0                  3 
         sitkInt16         sitkUInt32          sitkInt32         sitkUInt64 
                 2                  5                  4                  7 
         sitkInt64        sitkFloat32        sitkFloat64 sitkComplexFloat32 
                 6                  8                  9                 10 
sitkComplexFloat64    sitkVectorUInt8     sitkVectorInt8   sitkVectorUInt16 
                11                 13                 12                 15 
   sitkVectorInt16   sitkVectorUInt32    sitkVectorInt32   sitkVectorUInt64 
                14                 17                 16                 19 
   sitkVectorInt64  sitkVectorFloat32  sitkVectorFloat64     sitkLabelUInt8 
                18                 20                 21                 22 
   sitkLabelUInt16    sitkLabelUInt32    sitkLabelUInt64 
                23                 24                 25

It isn't usually necessary to access the enumeration classes directly, but it can be handy to know how when debugging.

Lets start querying the images we've created. The `$` operator is used to access object methods:

In [7]:
imA$GetSize()
imC$GetSpacing()


[1] 256 128  64

[1] 1 1

We can change the spacing (voxel size):

In [8]:
imC$SetSpacing(c(2, 0.25))
imC$GetSpacing()

NULL

[1] 2.00 0.25

There are shortcuts for accessing a single dimension:

In [11]:
imC$GetWidth()
imA$GetHeight()
imA$GetDepth()
imC$GetDepth()

[1] 32

[1] 128

[1] 64

[1] 0

The depth refers to the z dimension. A colour or vector image has an additional "dimension":

In [12]:
imRGB$GetNumberOfComponentsPerPixel()

[1] 3

The list of available methods can be retrieved using the following. There are many functions for accessing pixels in a type dependent way, which don't need to be accessed directly.


In [17]:
getMethod('$', class(imA))

An object of class “signature”
An object of class “signature”
An object of class “signature”
An object of class “signature”


function (x, name) 
{
    accessorFuns = list(`__seteq__` = Image___seteq__, GetITKBase = Image_GetITKBase, 
        GetPixelID = Image_GetPixelID, GetPixelIDValue = Image_GetPixelIDValue, 
        GetDimension = Image_GetDimension, GetNumberOfComponentsPerPixel = Image_GetNumberOfComponentsPerPixel, 
        GetOrigin = Image_GetOrigin, SetOrigin = Image_SetOrigin, 
        GetSpacing = Image_GetSpacing, SetSpacing = Image_SetSpacing, 
        GetDirection = Image_GetDirection, SetDirection = Image_SetDirection, 
        TransformIndexToPhysicalPoint = Image_TransformIndexToPhysicalPoint, 
        TransformPhysicalPointToIndex = Image_TransformPhysicalPointToIndex, 
        TransformPhysicalPointToContinuousIndex = Image_TransformPhysicalPointToContinuousIndex, 
        TransformContinuousIndexToPhysicalPoint = Image_TransformContinuousIndexToPhysicalPoint, 
        GetSize = Image_GetSize, GetHeight = Image_GetHeight, 
        GetWidth = Image_GetWidth, GetDepth = Image_GetDepth, 
        CopyInformation = Image_CopyInformation, GetMetaDataKeys = Image_GetMetaDataKeys, 
        HasMetaDataKey = Image_HasMetaDataKey, GetMetaData = Image_GetMetaData, 
        GetPixelIDTypeAsString = Image_GetPixelIDTypeAsString, 
        ToString = Image_ToString, GetPixelAsInt8 = Image_GetPixelAsInt8, 
        GetPixelAsUInt8 = Image_GetPixelAsUInt8, GetPixelAsInt16 = Image_GetPixelAsInt16, 
        GetPixelAsUInt16 = Image_GetPixelAsUInt16, GetPixelAsInt32 = Image_GetPixelAsInt32, 
        GetPixelAsUInt32 = Image_GetPixelAsUInt32, GetPixelAsInt64 = Image_GetPixelAsInt64, 
        GetPixelAsUInt64 = Image_GetPixelAsUInt64, GetPixelAsFloat = Image_GetPixelAsFloat, 
        GetPixelAsDouble = Image_GetPixelAsDouble, GetPixelAsVectorInt8 = Image_GetPixelAsVectorInt8, 
        GetPixelAsVectorUInt8 = Image_GetPixelAsVectorUInt8, 
        GetPixelAsVectorInt16 = Image_GetPixelAsVectorInt16, 
        GetPixelAsVectorUInt16 = Image_GetPixelAsVectorUInt16, 
        GetPixelAsVectorInt32 = Image_GetPixelAsVectorInt32, 
        GetPixelAsVectorUInt32 = Image_GetPixelAsVectorUInt32, 
        GetPixelAsVectorInt64 = Image_GetPixelAsVectorInt64, 
        GetPixelAsVectorUInt64 = Image_GetPixelAsVectorUInt64, 
        GetPixelAsVectorFloat32 = Image_GetPixelAsVectorFloat32, 
        GetPixelAsVectorFloat64 = Image_GetPixelAsVectorFloat64, 
        GetPixelAsComplexFloat32 = Image_GetPixelAsComplexFloat32, 
        GetPixelAsComplexFloat64 = Image_GetPixelAsComplexFloat64, 
        SetPixelAsInt8 = Image_SetPixelAsInt8, SetPixelAsUInt8 = Image_SetPixelAsUInt8, 
        SetPixelAsInt16 = Image_SetPixelAsInt16, SetPixelAsUInt16 = Image_SetPixelAsUInt16, 
        SetPixelAsInt32 = Image_SetPixelAsInt32, SetPixelAsUInt32 = Image_SetPixelAsUInt32, 
        SetPixelAsInt64 = Image_SetPixelAsInt64, SetPixelAsUInt64 = Image_SetPixelAsUInt64, 
        SetPixelAsFloat = Image_SetPixelAsFloat, SetPixelAsDouble = Image_SetPixelAsDouble, 
        SetPixelAsVectorInt8 = Image_SetPixelAsVectorInt8, SetPixelAsVectorUInt8 = Image_SetPixelAsVectorUInt8, 
        SetPixelAsVectorInt16 = Image_SetPixelAsVectorInt16, 
        SetPixelAsVectorUInt16 = Image_SetPixelAsVectorUInt16, 
        SetPixelAsVectorInt32 = Image_SetPixelAsVectorInt32, 
        SetPixelAsVectorUInt32 = Image_SetPixelAsVectorUInt32, 
        SetPixelAsVectorInt64 = Image_SetPixelAsVectorInt64, 
        SetPixelAsVectorUInt64 = Image_SetPixelAsVectorUInt64, 
        SetPixelAsVectorFloat32 = Image_SetPixelAsVectorFloat32, 
        SetPixelAsVectorFloat64 = Image_SetPixelAsVectorFloat64, 
        SetPixelAsComplexFloat32 = Image_SetPixelAsComplexFloat32, 
        SetPixelAsComplexFloat64 = Image_SetPixelAsComplexFloat64, 
        MakeUnique = Image_MakeUnique)
    idx = pmatch(name, names(accessorFuns))
    if (is.na(idx)) 
        return(callNextMethod(x, name))
    f = accessorFuns[[idx]]
    function(...) {
        f(x, ...)
    }
}
<environment: namespace:SimpleITK>
attr(,"target")
                     